# Initial Experimentation

## Web References

### Bitcoin Address

- [Bitcoin Address](https://learnmeabitcoin.com/technical/address)
- [Bitcoin Checksum](https://learnmeabitcoin.com/technical/checksum)
- [How do I check the checksum of a Bitcoin address?](https://bitcoin.stackexchange.com/questions/32353/how-do-i-check-the-checksum-of-a-bitcoin-address)
- [Base58Check encoding](https://en.bitcoin.it/wiki/Base58Check_encoding#Creating_a_Base58Check_string)

In [16]:
import unittest
import hashlib
import copy
from __future__ import annotations
from cryptography.hazmat.primitives import hashes
from cryptography.hazmat.primitives.serialization import load_der_public_key, load_der_private_key, Encoding, PublicFormat, PrivateFormat
from cryptography.hazmat.primitives.asymmetric import ec
from cryptography.hazmat.primitives.asymmetric import utils
from cryptography.exceptions import InvalidSignature

## Initial Unit tests

In [17]:
class TestTransaction(unittest.TestCase):
    def setUp(self):
        # create a new private key for testing
        self.private_key = ec.generate_private_key(ec.SECP256K1)
        self.recipient_hash = bytes.fromhex("3df8f04b3c159fdc6631c4b8b0874940344d173d")

        # create the sample transaction
        self.tx = Transaction(
            sender_hash=bytes.fromhex("3df8f04b3c159fdc6631c4b8b0874940344d173d"),
            recipient_hash=bytes.fromhex("5c1499a0484ace2f731b0afb83241e15f0e168ca"),
            sender_public_key=bytes.fromhex("3056301006072a8648ce3d020106052b8104000a" +
                                            "03420004886ed03cb7ffd4cbd95579ea2e202f1d" +
                                            "b29afc3bf5d7c2c34a34701bbb0685a7b535f1e6" +
                                            "31373afe8d1c860a9ac47d8e2659b74d437435b0" +
                                            "5f2c55bf3f033ac1"),
            amount=10,
            fee=2,
            nonce=5,
            signature=bytes.fromhex("3046022100f9c076a72a2341a1b8cb68520713e1" +
                                    "2f173378cf78cf79c7978a2337fbad141d022100" +
                                    "ec27704d4d604f839f99e62c02e65bf60cc93ae1"
                                    "735c1ccf29fd31bd3c5a40ed"),
            txid=bytes.fromhex("ca388e0890b71bd1775460d478f26af3776c9b4f" +
                                "6c2b936e1e788c5c87657bc3"))

    def create_test_transaction(self, amount:int, fee:int, nonce:int):
        """
        Create a test transaction.
        """
        return Transaction.create_signed_transaction(
            sender_private_key=self.private_key,
            recipient_hash=self.recipient_hash,
            amount=amount,
            fee=fee,
            nonce=nonce)

    def test_create_signed_transaction(self):
        """
        Test the creation of a new signed transaction.
        """
        # create a new transaction
        tx = Transaction.create_signed_transaction(
            sender_private_key=self.private_key,
            recipient_hash=self.recipient_hash,
            amount=33,
            fee=6,
            nonce=15)

        # validate the transaction
        tx.verify(300, 14)

    def test_verify_sender_hash(self):
        """
        Test that the sender_hash is verified correctly.
        """
        # test a valid hash
        self.tx.sender_hash = bytes.fromhex("3df8f04b3c159fdc6631c4b8b0874940344d173d")
        self.tx.verify(
            sender_balance=100,
            sender_previous_nonce=4)

        # test a sender_hash that do not match the public key
        self.tx.sender_hash = bytes.fromhex("5c1499a0484ace2f731b0afb83241e15f0e168ca")
        with self.assertRaisesRegex(ValueError, 'The sender hash is not computed correctly'):
            self.tx.verify(
                sender_balance=100,
                sender_previous_nonce=4)

        # test an invalid hash
        self.tx.sender_hash = bytearray(b'thequickbrownfox')
        with self.assertRaisesRegex(ValueError, 'Invalid sender hash'):
            self.tx.verify(
                sender_balance=100,
                sender_previous_nonce=4)

    def test_verify_recipient_hash(self):
        """
        Test that the recipient_hash is verified correctly.
        """
        # test a valid hash
        self.tx.recipient_hash = bytes.fromhex("5c1499a0484ace2f731b0afb83241e15f0e168ca")
        self.tx.verify(
            sender_balance=100,
            sender_previous_nonce=4)

        # test an invalid hash
        self.tx.recipient_hash = bytearray(b'thequickbrownfox')
        with self.assertRaisesRegex(ValueError, 'Invalid recipient hash'):
            self.tx.verify(
                sender_balance=100,
                sender_previous_nonce=4)

    def test_verify_amount(self):
        """
        Test that the amount is verified correctly.
        """
        # test a valid amount
        with self.subTest('Test for a valid amount'):
            tx = self.create_test_transaction(amount=50, fee=2, nonce=5)
            tx.verify(
                sender_balance=100,
                sender_previous_nonce=4)

        with self.subTest('The amount should be a whole number'):
            with self.assertRaisesRegex(ValueError, 'The amount should be a whole number'):
                tx = self.create_test_transaction(amount=50.1, fee=2, nonce=5)
                tx.verify(
                    sender_balance=100,
                    sender_previous_nonce=4)

        with self.subTest('The amount should be between 1 and sender_balance'):
            with self.assertRaisesRegex(ValueError, 'Invalid amount'):
                tx = self.create_test_transaction(amount=0, fee=2, nonce=5)
                tx.verify(
                    sender_balance=100,
                    sender_previous_nonce=4)

            with self.assertRaisesRegex(ValueError, 'Invalid amount'):
                tx = self.create_test_transaction(amount=-10, fee=2, nonce=5)
                tx.verify(
                    sender_balance=100,
                    sender_previous_nonce=4)

            with self.assertRaisesRegex(ValueError, 'Insufficient funds'):
                tx = self.create_test_transaction(amount=101, fee=2, nonce=5)
                tx.verify(
                    sender_balance=100,
                    sender_previous_nonce=4)

    def test_verify_fee(self):
        """
        Test that the fee is verified correctly.
        """
        with self.subTest('Test for a valid fee'):
            tx = self.create_test_transaction(amount=50, fee=5, nonce=5)
            tx.verify(
                sender_balance=100,
                sender_previous_nonce=4)

        with self.subTest('The fee can be zero'):
            tx = self.create_test_transaction(amount=50, fee=0, nonce=5)
            tx.verify(
                sender_balance=100,
                sender_previous_nonce=4)

        with self.subTest('The fee should be a whole number'):
            with self.assertRaisesRegex(ValueError, 'The fee should be a whole number'):
                tx = self.create_test_transaction(amount=50, fee=10.1, nonce=5)
                tx.verify(
                    sender_balance=100,
                    sender_previous_nonce=4)

        with self.subTest('The fee should be between 0 and amount'):
            with self.assertRaisesRegex(ValueError, 'Invalid fee'):
                tx = self.create_test_transaction(amount=50, fee=-10, nonce=5)
                tx.verify(
                    sender_balance=100,
                    sender_previous_nonce=4)

        with self.assertRaisesRegex(ValueError, 'The fee should be less than the amount'):
            tx = self.create_test_transaction(amount=50, fee=51, nonce=5)
            tx.verify(
                sender_balance=100,
                sender_previous_nonce=4)

    def test_verify_nonce(self):
        """
        Verify that the nonce is a valid value.
        """
        tx = self.create_test_transaction(amount=50, fee=5, nonce=10)

        # test a valid nonce
        with self.subTest('Test for a valid fee'):
            tx.verify(
                sender_balance=100,
                sender_previous_nonce=9)

        with self.subTest('The nonce should be sender_previous_nonce + 1'):        
            with self.assertRaisesRegex(ValueError, 'Invalid nonce'):
                tx.verify(
                    sender_balance=100,
                    sender_previous_nonce=8)

            with self.assertRaisesRegex(ValueError, 'Invalid nonce'):
                tx.verify(
                    sender_balance=100,
                    sender_previous_nonce=11)

    def test_create_txid(self):
        """
        Test Create a transaction id.
        """
        self.assertEqual(self.tx.create_txid(), self.tx.txid)

        # the transaction should not match if one of the values differ
        self.tx.fee = 3
        self.assertNotEqual(self.tx.create_txid(), self.tx.txid)

    def test_verify_txid(self):
        """
        Test that the txid is computed correctly.
        """
        # create a valid transaction
        with self.subTest('Test for a valid transaction'):
            tx = self.create_test_transaction(amount=50, fee=2, nonce=5)
            tx.verify(
                sender_balance=100,
                sender_previous_nonce=4)

        with self.subTest('Test with a different sender hash'):
            with self.assertRaisesRegex(ValueError, 'The transaction ID is invalid'):
                tx_invalid = copy.deepcopy(tx)
                tx_invalid.sender_hash = self.tx.sender_hash
                tx_invalid.sender_public_key = self.tx.sender_public_key

                tx_invalid.verify(
                    sender_balance=100,
                    sender_previous_nonce=4)

        with self.subTest('Test with a different recipient hash'):
            with self.assertRaisesRegex(ValueError, 'The transaction ID is invalid'):
                tx_invalid = copy.deepcopy(tx)
                tx_invalid.recipient_hash = self.tx.recipient_hash

                tx_invalid.verify(
                    sender_balance=100,
                    sender_previous_nonce=4)                    

        with self.subTest('Test with a different amount'):
            with self.assertRaisesRegex(ValueError, 'The transaction ID is invalid'):
                tx_invalid = copy.deepcopy(tx)
                tx_invalid.amount = tx_invalid.amount + 1

                tx_invalid.verify(
                    sender_balance=100,
                    sender_previous_nonce=4)

        with self.subTest('Test with a different fee'):
            with self.assertRaisesRegex(ValueError, 'The transaction ID is invalid'):
                tx_invalid = copy.deepcopy(tx)
                tx_invalid.fee = tx_invalid.fee + 1

                tx_invalid.verify(
                    sender_balance=100,
                    sender_previous_nonce=4)      

        with self.subTest('Test with a different signature'):
            with self.assertRaisesRegex(ValueError, 'The transaction ID is invalid'):
                tx_invalid = copy.deepcopy(tx)
                tx_invalid.signature = self.tx.signature

                tx_invalid.verify(
                    sender_balance=100,
                    sender_previous_nonce=4)

## Transaction Class

In [18]:
class Transaction:
    def __init__(self, sender_hash:bytes, recipient_hash:bytes, sender_public_key:bytes, amount:int, fee:int, nonce:int, signature, txid):
        self.sender_hash = sender_hash
        self.recipient_hash = recipient_hash
        self.sender_public_key = sender_public_key
        self.amount = amount
        self.fee = fee
        self.nonce = nonce
        self.signature = signature
        self.txid = txid
        #self.hash = self.calculate_hash()

    def verify(self, sender_balance:int, sender_previous_nonce:int) -> bool:
        # sender and recipient validation
        if len(self.sender_hash) != 20:
            raise ValueError('Invalid sender hash')
        if len(self.recipient_hash) != 20:
            raise ValueError('Invalid recipient hash')

        # validate the sender hash
        if self.sender_hash != hashlib.sha1(self.sender_public_key).digest():
            raise ValueError('The sender hash is not computed correctly')

        # amount validation
        if self.amount < 1:
            raise ValueError('Invalid amount')
        if self.amount > sender_balance:
            raise ValueError('Insufficient funds')
        if float(self.amount).is_integer() == False:
            raise ValueError('The amount should be a whole number')

        # fee validation
        if self.fee < 0:
            raise ValueError('Invalid fee')
        if self.fee > self.amount:
            raise ValueError('The fee should be less than the amount')
        if float(self.fee).is_integer() == False:
            raise ValueError('The fee should be a whole number')

        # nonce validation
        if self.nonce <= sender_previous_nonce:
            raise ValueError('Invalid nonce')
        if self.nonce > sender_previous_nonce + 1:
            raise ValueError('Invalid nonce')

        # verify the transaction id
        if self.txid != self.create_txid():
            raise ValueError('The transaction ID is invalid')

        # validate the signature
        try:
            sender_public_key = load_der_public_key(self.sender_public_key)
            sender_public_key.verify(
                self.signature, 
                self.create_signature_hash(), 
                ec.ECDSA(utils.Prehashed(hashes.SHA256())))
        except InvalidSignature as e:
            raise ValueError('The signature is invalid')

        return True
        
    def create_txid(self) -> bytes:
        """
        Create a hash of the transaction.
        """
        # create a hash of the transaction
        txid = hashes.Hash(hashes.SHA256())
        txid.update(self.sender_hash)
        txid.update(self.recipient_hash)
        txid.update(self.sender_public_key)
        txid.update(self.amount.to_bytes(8, byteorder = 'little', signed = False))
        txid.update(self.fee.to_bytes(8, byteorder = 'little', signed = False))
        txid.update(self.nonce.to_bytes(8, byteorder = 'little', signed = False))
        txid.update(self.signature)

        return txid.finalize()

    def create_signature_hash(self) -> bytes:
        """
        Create a hash of the recipient transaction data.
        """
        recipient_hash = hashes.Hash(hashes.SHA256())
        recipient_hash.update(self.recipient_hash)
        recipient_hash.update(self.amount.to_bytes(8, byteorder = 'little', signed = False))
        recipient_hash.update(self.fee.to_bytes(8, byteorder = 'little', signed = False))
        recipient_hash.update(self.nonce.to_bytes(8, byteorder = 'little', signed = False))
        
        return recipient_hash.finalize()

    @staticmethod
    def create_signed_transaction(sender_private_key:ec.EllipticCurvePrivateKey, recipient_hash:bytes, amount:int, fee:int, nonce:int) -> Transaction:
        """
        Create a signed transaction.
        """
        # validate the amount
        if amount < 0:
            raise ValueError('Invalid amount')
        if float(amount).is_integer() == False:
            raise ValueError('The amount should be a whole number')

        # fee validation
        if fee < 0:
            raise ValueError('Invalid fee')
        if float(fee).is_integer() == False:
            raise ValueError('The fee should be a whole number')

        # get the public key
        public_key = sender_private_key.public_key()
        sender_public_key = public_key.public_bytes(
                encoding=Encoding.DER,
                format=PublicFormat.SubjectPublicKeyInfo)

        # create the new transaction
        tx = Transaction(
            sender_hash = hashlib.sha1(sender_public_key).digest(),
            recipient_hash = recipient_hash,
            sender_public_key = sender_public_key,
            amount = amount,
            fee = fee,
            nonce = nonce,
            signature = None,
            txid = None)

        # sign the transaction
        tx.signature = sender_private_key.sign(
            tx.create_signature_hash(),
            ec.ECDSA(utils.Prehashed(hashes.SHA256()))
        )

        # create the transaction id
        tx.txid = tx.create_txid()

        return tx


unittest.main(argv=['ignored', '-v'], exit=False)

test_create_signed_transaction (__main__.TestTransaction)
Test the creation of a new signed transaction. ... ok
test_create_txid (__main__.TestTransaction)
Test Create a transaction id. ... ok
test_verify_amount (__main__.TestTransaction)
Test that the amount is verified correctly. ... ok
test_verify_fee (__main__.TestTransaction)
Test that the fee is verified correctly. ... ok
test_verify_nonce (__main__.TestTransaction)
Verify that the nonce is a valid value. ... ok
test_verify_recipient_hash (__main__.TestTransaction)
Test that the recipient_hash is verified correctly. ... ok
test_verify_sender_hash (__main__.TestTransaction)
Test that the sender_hash is verified correctly. ... ok
test_verify_txid (__main__.TestTransaction)
Test that the txid is computed correctly. ... ok
test_create_signed_transaction (__main__.TransactionTesting)
Generate a private key using ec.generate_private_key(ec.SECP256K1) . Call ... ok
test_invalid_new_signature (__main__.TransactionTesting)
Generate two pr

## Mandatory Unit Tests

In [19]:
class TransactionTesting(unittest.TestCase):
    def create_valid_transaction(self, sender_private_key:ec.EllipticCurvePrivateKey=None, amount:int=10, fee:int=1, nonce:int=2) -> Transaction:
        """
        Create a valid transaction.
        """
        # create a private keys
        if sender_private_key == None:
            sender_private_key = ec.generate_private_key(ec.SECP256K1())

        recipient_private_key = ec.generate_private_key(ec.SECP256K1())

        # get the recipient hash
        recipient_hash = hashlib.sha1(
            recipient_private_key.public_key().public_bytes(
                encoding=Encoding.DER,
                format=PublicFormat.SubjectPublicKeyInfo)).digest()   

        # create the transaction
        return Transaction.create_signed_transaction(
            sender_private_key=sender_private_key,
            recipient_hash=recipient_hash,
            amount=amount,
            fee=fee,
            nonce=nonce)

    def test_create_signed_transaction(self):
        """
        Generate a private key using ec.generate_private_key(ec.SECP256K1) . Call
        create_signed_transaction to make a test transaction. Check that the
        transaction.verify call succeeds (when provided with a sender_balance which is
        sufficiently high and sender_previous_nonce = transaction.nonce - 1 ).
        """
        # create a sender private
        sender_private_key = ec.generate_private_key(ec.SECP256K1())

        # create a transaction
        tx = self.create_valid_transaction(sender_private_key=sender_private_key)

        # verify the transaction
        self.assertTrue(
            tx.verify(
                sender_balance=100,
                sender_previous_nonce=tx.nonce - 1))

    def test_verify_txid(self):
        """
        Generate a valid transaction, check that modifying any of the fields causes
        transaction.verify to raise an exception due to an invalid txid.        
        """
        # create a valid transaction
        tx = self.create_valid_transaction()
        tx.verify(sender_balance=100, sender_previous_nonce=tx.nonce-1)

        with self.assertRaisesRegex(ValueError, 'The sender hash is not computed correctly'):
            tx_invalid = copy.deepcopy(tx)
            tx_invalid.sender_hash = bytes.fromhex("3df8f04b3c159fdc6631c4b8b0874940344d173d")
            tx_invalid.verify(sender_balance=100, sender_previous_nonce=tx_invalid.nonce-1)

        with self.assertRaisesRegex(ValueError, 'The transaction ID is invalid'):
            tx_invalid.sender_public_key=bytes.fromhex("3056301006072a8648ce3d020106052b8104000a" +
                                            "03420004886ed03cb7ffd4cbd95579ea2e202f1d" +
                                            "b29afc3bf5d7c2c34a34701bbb0685a7b535f1e6" +
                                            "31373afe8d1c860a9ac47d8e2659b74d437435b0" +
                                            "5f2c55bf3f033ac1")
            tx_invalid.verify(sender_balance=100, sender_previous_nonce=tx_invalid.nonce-1)            

        with self.assertRaisesRegex(ValueError, 'The transaction ID is invalid'):
            tx_invalid = copy.deepcopy(tx)
            tx_invalid.recipient_hash = bytes.fromhex("5c1499a0484ace2f731b0afb83241e15f0e168ca")
            tx_invalid.verify(sender_balance=100, sender_previous_nonce=tx_invalid.nonce-1)

        with self.assertRaisesRegex(ValueError, 'The transaction ID is invalid'):
            tx_invalid = copy.deepcopy(tx)
            tx_invalid.amount = tx_invalid.amount + 1
            tx_invalid.verify(sender_balance=100, sender_previous_nonce=tx_invalid.nonce-1)

        with self.assertRaisesRegex(ValueError, 'The transaction ID is invalid'):
            tx_invalid = copy.deepcopy(tx)
            tx_invalid.fee = tx_invalid.fee + 1
            tx_invalid.verify(sender_balance=100, sender_previous_nonce=tx_invalid.nonce-1)

        with self.assertRaisesRegex(ValueError, 'The transaction ID is invalid'):
            tx_invalid = copy.deepcopy(tx)
            tx_invalid.nonce = tx_invalid.nonce + 1
            tx_invalid.verify(sender_balance=100, sender_previous_nonce=tx_invalid.nonce-1)


        with self.assertRaisesRegex(ValueError, 'The transaction ID is invalid'):
            tx_invalid = copy.deepcopy(tx)
            tx_invalid.signature = bytes.fromhex("3046022100f9c076a72a2341a1b8cb68520713e1" +
                                    "2f173378cf78cf79c7978a2337fbad141d022100" +
                                    "ec27704d4d604f839f99e62c02e65bf60cc93ae1"
                                    "735c1ccf29fd31bd3c5a40ed")
            tx_invalid.verify(sender_balance=100, sender_previous_nonce=tx_invalid.nonce-1)       

    def test_invalid_signature(self):
        """"
        Generate a valid transaction, change the amount field, regenerate the txid so it is valid
        again. Check that transaction.verify raises an exception due to an invalid signature.
        """
        # create a valid transaction
        tx = self.create_valid_transaction(amount=50)
        tx.verify(sender_balance=100, sender_previous_nonce=tx.nonce-1)

        # change the amount and regenerate the txid
        tx.amount = 5
        tx.txid = tx.create_txid()

        with self.assertRaisesRegex(ValueError, 'The signature is invalid'):
            tx.verify(sender_balance=100, sender_previous_nonce=tx.nonce-1)

    def test_invalid_sender_balance(self):
        """
        Generate a valid transaction, check that transaction.verify raises an exception if either
        the sender_balance is too low or sender_previous_nonce is incorrect.
        """
        # create a valid transaction
        tx = self.create_valid_transaction(amount=50)
        tx.verify(sender_balance=100, sender_previous_nonce=tx.nonce-1)

        # verify with a balance that is too low
        with self.assertRaisesRegex(ValueError, 'Insufficient funds'):
            tx.verify(sender_balance=10, sender_previous_nonce=tx.nonce-1)

        # verify with a previous nonce that is incorrect
        with self.assertRaisesRegex(ValueError, 'Invalid nonce'):
            tx.verify(sender_balance=100, sender_previous_nonce=55)
        

    def test_invalid_new_signature(self):
        """
        Generate two private keys, A and B . Use A to generate a valid transaction. Replace the
        signature with a signature created using B . Regenerate the txid and confirm that
        transaction.verify fails with an invalid signature.
        """

    def test_model_transaction(self):
        """
        Check that the following transaction verifies successfully (when using sender_balance = 20 ,
        sender_previous_nonce = 4 )
        """

unittest.main(argv=['ignored', '-v', 'TransactionTesting'], exit=False)


test_create_signed_transaction (__main__.TransactionTesting)
Generate a private key using ec.generate_private_key(ec.SECP256K1) . Call ... ok
test_invalid_new_signature (__main__.TransactionTesting)
Generate two private keys, A and B . Use A to generate a valid transaction. Replace the ... ok
test_invalid_sender_balance (__main__.TransactionTesting)
Generate a valid transaction, check that transaction.verify raises an exception if either ... ok
test_invalid_signature (__main__.TransactionTesting)
" ... ok
test_model_transaction (__main__.TransactionTesting)
Check that the following transaction verifies successfully (when using sender_balance = 20 , ... ok
test_verify_txid (__main__.TransactionTesting)
Generate a valid transaction, check that modifying any of the fields causes ... ok

----------------------------------------------------------------------
Ran 6 tests in 0.041s

OK


In [20]:
#%history -g